In [12]:
%matplotlib inline

from dask.distributed import Client, progress, LocalCluster
#from dask_kubernetes import KubeCluster
import xarray as xr
#import hvplot.xarray
#import holoviews as hv
#import geoviews as gv
#from holoviews.operation.datashader import datashade, shade, dynspread, rasterize

#import cartopy.crs as ccrs
#import s3fs
import os
import numpy as np
#import intake
import rasterio 
os.environ['CURL_CA_BUNDLE']='/etc/ssl/certs/ca-certificates.crt'

ModuleNotFoundError: No module named 'intake'

# 0: Threshold and Tile Ground Truth Raster

We're working with large files (> ~4gb), so we'll use xarray and either a local or Kubernetes cluster. 

(*Note: remote cloud-optimized GeoTIFFs are required to use the Kubernetes cluster, which doesn't have access to a shared local file system*)

## Cluster Setup

In [7]:
#cluster = KubeCluster.from_yaml("/home/jovyan/worker-template.yaml")
cluster = LocalCluster()

In [8]:
cluster.scale(4);

In [9]:
cluster

In [10]:
client = Client(cluster)

## Load Data + Compute Threshold
We're using a data catalog that contains 3 meter ASO data stored in cloud optimized GeoTIFFs on Amazon S3. 

In [11]:
cat = intake.open_catalog("../intake/aso_raw.yaml")

NameError: name 'intake' is not defined

In [7]:
ds = cat.ASO_3M_SD_USCASJ_20180601.to_dask()

In [8]:
threshold = ds.sel(band=1) >= 0.5
nodata = xr.full_like(ds, np.nan)
masked = xr.where(ds.sel(band = 1) != ds.nodatavals[0], threshold, nodata)

## Save to S3

In [10]:
fs = s3fs.S3FileSystem()

In [77]:
values = masked.persist()

In [39]:
fs.ls('planet-snowcover-snow/ASO_3M_SD_USCASJ_20180601')

['planet-snowcover-snow/ASO_3M_SD_USCASJ_20180601/ASO_3M_SD_USCASJ_20180601.tif',
 'planet-snowcover-snow/ASO_3M_SD_USCASJ_20180601/ASO_3M_SD_USCASJ_20180601_COG.tif',
 'planet-snowcover-snow/ASO_3M_SD_USCASJ_20180601/ASO_3M_SD_USCASJ_20180601_COG_FINAL.tif']

In [46]:
def to_tiff(data, fname, template=None, **kwargs):
    import numpy as np

    # check and promote the number of dimentio(1)ns for consistency
    nbands = data.ndim
    if 2 == nbands:
     # expand the array so that it is least 3D (ie stacks ofsurfaces)
        import numpy as np
        data = np.expand_dims(data,axis=0)
    elif 3 != nbands: #  nothing to do if it is already 3D
        print("Error: to_tiff can only currently deal with 2D and 3D data")
        return

    profile = {}
    tags = {}
    tmpl = None
    if template:
        tmpl = rasterio.open(template,'r')
        profile = tmpl.profile.copy()
        tags = tmpl.tags()

    # the metadata should be appended.  Cache here to
    # simplify variable replacement below.
    meta = {}
    if 'meta' in profile:
        meta.update(profile['meta'])
    if 'meta' in kwargs:
        meta.update(kwargs['meta'])

    # overwrite anything inheritied from the template with
    # user supplied args
    profile.update(kwargs)

    # overwrite bits that write the array as geotiff and
    # save the cached metadata
    profile['driver'] = 'GTiff'
    profile['count'] = data.shape[0]
    profile['width'] = data.shape[2]
    profile['height'] = data.shape[1]
    profile['meta'] = meta

    if 'dtype' not in profile:
        profile['dtype'] = type(data[0,0,0])

    # if you do not remove the previously associated .xml file,
    # then the tags and metadata can get corrupted.
    try:
        os.remove(fname)
        os.remove(fname+".xml")
    except:
         pass

    # now create and save the array to a file
    with rasterio.open(fname,'w',**profile) as out:
        for b in range(data.shape[0]):
            #print("\nprocessing band %d"%(b+1))
            out.write(data[b].astype(profile['dtype']), b+1)

         # caluclate the stats for each band
         # not sure what the proper name for per band stats is in QGIS
#          stats = {
#                  'STATISTICS_MINIMUM': np.nanmin(data[b]),
#                  'STATISTICS_MAXIMUM': np.nanmax(data[b]),
#                  'STATISTICS_MEAN': np.nanmean(data[b]),
#                  'STATISTICS_STDDEV': np.nanstd(data[b])}
#          out.update_tags(b+1,**stats)
         #print("  stats= %s"%str(stats))

     # now calculate the stats across all the bands
#      stats = {
#              'STATISTICS_MINIMUM': np.nanmin(data),
#              'STATISTICS_MAXIMUM': np.nanmax(data),
#              'STATISTICS_MEAN': np.nanmean(data),
#              'STATISTICS_STDDEV': np.nanstd(data)}

#      out.update_tags(**tags)
#      if 'tags' in kwargs:
#          out.update_tags(**kwargs['tags'])

#      out.update_tags(**stats)
     #print("\n  overall stats= %s\n"%str(stats))

    del tmpl

In [54]:
to_tiff(data=values, template=cat.ASO_3M_SD_USCASJ_20180601.describe_open()['args']['urlpath'], fname = '/tmp/testout.tif')

ValueError: blockxsize exceeds raster width.

In [61]:
zarr = values.to_dataset('masked').to_zarr()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: the order of the arguments on DataArray.to_dataset has changed; you now need to supply ``name`` as a keyword argument
  """Entry point for launching an IPython kernel.


In [70]:
mapping = s3fs.S3Map('planet-snowcover-snow/ASO_3M_SD_USCASJ_20180601/', s3=fs)

In [71]:
values.to_dataset('masked').to_zarr(store=mapping)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: the order of the arguments on DataArray.to_dataset has changed; you now need to supply ``name`` as a keyword argument
  """Entry point for launching an IPython kernel.


ValueError: array not found at path 'masked'

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [ ]:
(ds.sel(band = 1) != ds.nodatavals[0]).hvplot.quadmesh()